In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import timeit
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,BatchNormalization, Activation,Input,Lambda
from tensorflow.keras.activations import relu
from tensorflow.keras import regularizers
from tensorflow.keras.losses import mean_absolute_error
from tensorflow.keras.models import Model

In [2]:
# TODO: relu at output
# TODO: repeat X
# TODO: BN train mode
# TODO: remove baseline

In [3]:
batchSize = 2000
dateRange = (370,670)
weight = 1/np.arange(1000,700,-1)
weight = weight/np.sum(weight)

In [4]:
train1  = pd.read_csv('train_2.csv')
missing = train1.isnull()
nonMissingIndex = np.logical_not(missing).values
train1 = train1.fillna(0)
train1.iloc[:,1:] = train1.iloc[:,1:].astype(np.float32)
n = train1.shape[0]

In [5]:
a = train1.Page.str.split('mediawiki.org|wikimedia.org|wikipedia.org',expand=True)
b = np.where(a.loc[:,2].isnull(),a.loc[:,1],a.loc[:,2])
_,index_array = np.unique(b,return_inverse=True) # access & agent
website_array = [train1.Page.str.contains(web).values.astype(np.int8) \
                 for web in ['mediawiki.org','wikimedia.org','wikipedia.org']] # website
lang_array = train1.Page.str.extract('([a-z]{2}).wikipedia').fillna('nan').values.flatten()
_,lang_array = np.unique(lang_array,return_inverse=True) # language

In [6]:
Page_X = \
np.concatenate([OneHotEncoder(dtype=np.float32,sparse=False).fit_transform(lang_array[:,np.newaxis]),\
                OneHotEncoder(dtype=np.float32,sparse=False).fit_transform(index_array[:,np.newaxis]),\
                np.stack(website_array,1)],1)

In [7]:
date_index=pd.to_datetime(train1.columns[1:]).to_series().reset_index(drop=True)

'''def extract_date_info(timestamp,mss):
    return np.array([mss, timestamp.day, \
                     timestamp.dayofyear]+\
                    [1 if i==timestamp.year else 0 for i in [2015, 2016, 2017]]+\
                    [1 if i==timestamp.weekday() else 0 for i in range(7)]+\
                    [1 if i==timestamp.month else 0 for i in range(12)])'''

def extract_date_info(timestamp,mss):
    return np.array([mss, timestamp.day, \
                     timestamp.dayofyear,\
                     timestamp.year,\
                     timestamp.month,\
                     timestamp.is_month_start,\
                     timestamp.is_month_end]+\
                    [1 if i==timestamp.weekday() else 0 for i in range(7)])

In [8]:
d = extract_date_info(date_index[3],3).shape[0]

In [9]:
max_ = lambda df,start,interval: np.max(df.iloc[:,start-interval:start].values,1)
min_ = lambda df,start,interval: np.min(df.iloc[:,start-interval:start].values,1)
std_ = lambda df,start,interval: np.std(df.iloc[:,start-interval:start].values,1)
mean_ = lambda df,start,interval: np.mean(df.iloc[:,start-interval:start].values,1)
growth_ = lambda df,start,interval: np.mean(df.iloc[:,start-interval:start],1) -\
                                    np.mean(df.iloc[:,start-2*interval:start-interval],1)

fun_list = [max_,min_,std_,mean_,growth_]
interval_list = [14,30,60]
T = train1.shape[1]

In [10]:
def Generator(batchSize,dateRange,weight):
    while True:
        j = np.random.choice(range(*dateRange),p=weight)
        i = np.random.choice(range(j+2,j+64))
        index_ = np.random.choice(np.where(nonMissingIndex[:,i])[0],size=batchSize)
        temp = train1.iloc[index_]
        Lookback_X = np.stack([fun(temp,j,interval) for fun in fun_list for interval in interval_list],1)
        yield np.concatenate([Page_X[index_],\
                                Lookback_X,\
                                np.broadcast_to(extract_date_info(date_index[i-1],i-j),(batchSize,d)),\
                                temp.iloc[:,j-1:j].values,\
                                temp.iloc[:,i-90:i-89].values,\
                                temp.iloc[:,i-120:i-119].values,\
                                temp.iloc[:,i-180:i-179].values,\
                                temp.iloc[:,i-365:i-364].values],1).astype(np.float32),\
               temp.iloc[:,j-1].values[:,np.newaxis],\
               temp.iloc[:,i].values[:,np.newaxis]

In [11]:
temp = np.concatenate([next(Generator(batchSize,dateRange,weight))[0] for _ in range(2000)],0)

In [12]:
std_ = temp.std(0)
mean_ = temp.mean(0)

In [13]:
# np.savetxt('std_.txt',std_)
# np.savetxt('mean_.txt',mean_)

In [14]:
def Generator(batchSize,dateRange,weight):
    while True:
        j = np.random.choice(range(*dateRange),p=weight)
        i = np.random.choice(range(j+2,j+64))
        index_ = np.random.choice(np.where(nonMissingIndex[:,i])[0],size=batchSize)
        temp = train1.iloc[index_]
        Lookback_X = np.stack([fun(temp,j,interval) for fun in fun_list for interval in interval_list],1)
        yield ([(np.concatenate([Page_X[index_],\
                                Lookback_X,\
                                np.broadcast_to(extract_date_info(date_index[i-1],i-j),(batchSize,d)),\
                                temp.iloc[:,j-1:j].values,\
                                temp.iloc[:,i-90:i-89].values,\
                                temp.iloc[:,i-120:i-119].values,\
                                temp.iloc[:,i-180:i-179].values,\
                                temp.iloc[:,i-365:i-364].values],1).astype(np.float32) - mean_)/std_,\
                temp.iloc[:,j-1].values[:,np.newaxis]],\
                temp.iloc[:,i].values[:,np.newaxis])

In [15]:
def Generator_noBL(batchSize,dateRange,weight):
    while True:
        j = np.random.choice(range(*dateRange),p=weight)
        i = np.random.choice(range(j+2,j+64))
        index_ = np.random.choice(np.where(nonMissingIndex[:,i])[0],size=batchSize)
        temp = train1.iloc[index_]
        Lookback_X = np.stack([fun(temp,j,interval) for fun in fun_list for interval in interval_list],1)
        yield ((np.concatenate([Page_X[index_],\
                                Lookback_X,\
                                np.broadcast_to(extract_date_info(date_index[i-1],i-j),(batchSize,d)),\
                                temp.iloc[:,j-1:j].values,\
                                temp.iloc[:,i-90:i-89].values,\
                                temp.iloc[:,i-120:i-119].values,\
                                temp.iloc[:,i-180:i-179].values,\
                                temp.iloc[:,i-365:i-364].values],1).astype(np.float32) - mean_)/std_,\
                temp.iloc[:,i].values[:,np.newaxis])

In [16]:
downSampleRate = 0.25
j = T - 64
Lookback_X = np.stack([fun(train1,j,interval) for fun in fun_list for interval in interval_list],1)
testData = []
testMargain = []
for i in range(j+2,j+64):
    temp = np.concatenate([train1.iloc[:,i].values[:,np.newaxis],\
                           Page_X,\
                           np.broadcast_to(extract_date_info(date_index[i-1],i-j),(n,d)),\
                           Lookback_X,\
                           train1.iloc[:,j-1:j].values,\
                           train1.iloc[:,i-90:i-89].values,\
                           train1.iloc[:,i-120:i-119].values,\
                           train1.iloc[:,i-180:i-179].values,\
                           train1.iloc[:,i-365:i-364].values],1).astype(np.float32)
    nonmissing_index = np.logical_not(missing.iloc[:,i].values)
    random_index = np.random.rand(nonmissing_index.sum())<downSampleRate
    testData.append(temp[nonmissing_index][random_index])
    testMargain.append(train1.iloc[:,j-1].values[nonmissing_index][random_index])
testData = np.concatenate(testData,0)
testMargain = np.concatenate(testMargain,0)    

In [17]:
testY = testData[:,0][:,np.newaxis]
testX = testData[:,1:]
testMargain = testMargain[:,np.newaxis]
del testData

In [18]:
testX = (testX-mean_)/std_

In [141]:
depth = 2
d_X = 200
L2 = 1e-2
learningR = 1e-3
eps = 0.1
epoch = 500
monitor = 100

In [142]:
tf.reset_default_graph()
keras.backend.clear_session()

In [143]:
X_tf = Input(shape=(testX.shape[1],))
base_tf = Input(shape=(1,))

In [144]:
temp_tf = X_tf
for _ in range(depth):   
    temp_tf = Dense(d_X,kernel_regularizer=regularizers.l2(L2))(temp_tf)
    temp_tf = BatchNormalization()(temp_tf)
    #temp_tf = keras.layers.LeakyReLU()(temp_tf)
    temp_tf = keras.layers.PReLU()(temp_tf)

In [145]:
temp_tf = Dense(1,kernel_regularizer=regularizers.l2(L2))(temp_tf)
yhat_tf = Lambda(lambda x:x[0]+x[1])([temp_tf,base_tf])
#yhat_tf = Dense(1,kernel_regularizer=regularizers.l2(L2))(temp_tf)

In [146]:
model1 = Model([X_tf,base_tf],yhat_tf)
#model1 = Model(X_tf,yhat_tf)

In [147]:
def SMAPE_tf(y, yhat):
    summ = tf.abs(y) + tf.abs(yhat) + 1e-3
    return 200*tf.reduce_mean(tf.abs(y - yhat)/summ)

In [148]:
def SMAPE_train(y, yhat):
    summ = tf.abs(y) + tf.abs(yhat) + 1e-3
    return 200*tf.reduce_mean(tf.abs(y - yhat)/summ,-1)

In [149]:
def SMAPE_np(y, yhat):
    summ = np.abs(y) + np.abs(yhat) + 1e-3
    return 200*np.mean(np.abs(y - yhat)/summ)

In [150]:
#model1.compile(keras.optimizers.SGD(lr=learningR),keras.losses.MAE,[SMAPE_tf])
model1.compile(keras.optimizers.Adam(lr=learningR),SMAPE_train,[SMAPE_tf])
#model1.compile(keras.optimizers.SGD(lr=learningR),SMAPE_train,[SMAPE_tf])

In [151]:
model1.fit_generator(Generator(batchSize,dateRange,weight),100,10,1,validation_data=([testX,testMargain],testY))
#model1.fit_generator(Generator_noBL(batchSize,dateRange,weight),100,10,1,validation_data=(testX,testY))

Epoch 1/10
100/100 [==============================] - 4s 44ms/step - loss: 53.8141 - SMAPE_tf: 51.4777 - val_loss: 63.4331 - val_SMAPE_tf: 61.4671
Epoch 2/10
100/100 [==============================] - 3s 32ms/step - loss: 52.7499 - SMAPE_tf: 51.0083 - val_loss: 50.2388 - val_SMAPE_tf: 48.6949
Epoch 3/10
100/100 [==============================] - 3s 32ms/step - loss: 51.9566 - SMAPE_tf: 50.5619 - val_loss: 48.3480 - val_SMAPE_tf: 47.0883
Epoch 4/10
100/100 [==============================] - 3s 32ms/step - loss: 52.8319 - SMAPE_tf: 51.6829 - val_loss: 47.9964 - val_SMAPE_tf: 46.9517
Epoch 5/10
100/100 [==============================] - 3s 32ms/step - loss: 52.4825 - SMAPE_tf: 51.5248 - val_loss: 47.8038 - val_SMAPE_tf: 46.9311
Epoch 6/10
100/100 [==============================] - 3s 32ms/step - loss: 51.0700 - SMAPE_tf: 50.2695 - val_loss: 47.6383 - val_SMAPE_tf: 46.9059
Epoch 7/10
100/100 [==============================] - 3s 33ms/step - loss: 50.8483 - SMAPE_tf: 50.1713 - val_loss: 47.

Make Submission

In [ ]:
date_index = date_index.append(pd.Series(pd.date_range('2017-09-11','2017-11-13')))

In [ ]:
date_index = date_index.reset_index(drop=True)

In [ ]:
j = T
Lookback_X = np.stack([fun(train1,j,interval) for fun in fun_list for interval in interval_list],1)
testData = []
testMargain = []
mapping_index = []
for i in range(j+2,j+64):
    temp = np.concatenate([Page_X,\
                           np.broadcast_to(extract_date_info(date_index[i-1],i-j),(n,d)),\
                           Lookback_X,\
                           train1.iloc[:,j-1:j].values,\
                           train1.iloc[:,i-90:i-89].values,\
                           train1.iloc[:,i-120:i-119].values,\
                           train1.iloc[:,i-180:i-179].values,\
                           train1.iloc[:,i-365:i-364].values],1).astype(np.float32)

    testData.append(temp)
    testMargain.append(train1.iloc[:,j-1].values)
    mapping_index.append(train1.iloc[:,0].str.cat([str(date_index[i-1])[:10]]*n,'_'))
    
testData = np.concatenate(testData,0)
testMargain = np.concatenate(testMargain,0)
mapping_index = np.concatenate(mapping_index,0)

In [ ]:
# Data corruption when creating a a DMatrix with label being a non-contiguous ndarray #2554
dtest = xgb.DMatrix(np.ascontiguousarray(testData))
dtest.set_base_margin(np.ascontiguousarray(testMargain))

In [ ]:
yhat = np.maximum(model_gbm.predict(dtest,output_margin=True),0)

In [ ]:
yhat = pd.DataFrame({'Page':mapping_index,'Visits':yhat})

In [ ]:
key = pd.read_csv('key_2.csv')

In [ ]:
key = key.merge(yhat,on='Page')

In [ ]:
key.drop('Page',axis=1,inplace=True)

In [ ]:
key.to_csv('Submissions/gbm1.csv',index=False)